A sample CBF design is saved in a csv file. We first read in the data

In [1]:
import pandas as pd
test_cbf = pd.read_csv('cbf.csv', index_col=0, header=0).T
cbf_params = test_cbf.iloc[0]
# preview the data
cbf_params

num_bays           5.000000
num_stories        4.000000
num_frames         2.000000
S_1                1.185500
T_m                2.749952
k_ratio            5.739507
Q                  0.071810
moat_ampli         1.775365
RI                 1.445491
L_bay             30.000000
h_story           13.000000
S_s                2.281500
W              10687.500000
W_s             8437.500000
mu_1               0.023457
mu_2               0.086961
R_1               12.374762
R_2               51.674927
T_e                2.830994
k_e                0.012748
zeta_e             0.147725
D_m               23.755869
Name: 24, dtype: float64

In [2]:
# temporary fix to add loads to the Series
import numpy as np
cbf_params['w_fl'] = np.array([3., 3., 3., 3., 2.013])
cbf_params['P_lc'] = np.array([1800., 1800., 1800., 1800., 1208.])

# accessing a sample value
n_bays = int(cbf_params['num_bays'])
cbf_params

num_bays                                            5.0
num_stories                                         4.0
num_frames                                          2.0
S_1                                              1.1855
T_m                                            2.749952
k_ratio                                        5.739507
Q                                               0.07181
moat_ampli                                     1.775365
RI                                             1.445491
L_bay                                              30.0
h_story                                            13.0
S_s                                              2.2815
W                                               10687.5
W_s                                              8437.5
mu_1                                           0.023457
mu_2                                           0.086961
R_1                                           12.374762
R_2                                           51

# Support utilities

These are utility functions that we can use in the construction of the model later on.

## Descriptions

`bot_gp_coord` is a function that returns the x and z coordinate of a node attached to a bottom node of a brace (e.g. 2103, 2104, 2101, 2102). It takes the node number and figures out its location , then places it appropriately according the bay length and height. It also takes an argument `offset` which is how detached we put the node from its parent node. Here, we leave a default of 0.25, meaning "put the end of the gusset plate 25% of the brace length away from the beam-column joint". IMPORTANT: this function assumes that nodes xxx3 and xxx4 are of a brace going upwards and right (northeast), while xxx1 and xxx2 goes to the northwest. If you have a different numbering system, change the `goes_ne` variable accordingly.

`top_gp_coord` behaves similarly, assuming that nodes xxx1 and xxx5 connects southeast, while xxx2 and xxx6 connects southwest. The logic is handled in the if-else statement, which just checks if the last number is odd or even. Change your line accordingly if you have a different numbering system.

`mid_brace_coord` takes the mid-brace node's number and calculates its position with a camber designed to introduce initial imperfection into the brace. This helps enables its buckling behavior later on. We place the nodes with a default 0.1% camber, meaning the middle node lies 0.1% of the brace length away from the straight line.

IMPORTANT: my system numbers the middle node accordingly to the top node it's attached to. In other words, node 3117 is the  middle node extending southeast from 311. You will have to change this system. For example, for your mid brace node 2106, calculate which is the bottom node it connects to and which is the top node it connects to. Make the appropriate changes in lines `top_node =` and `bot_node = `.

In [ ]:
def bot_gp_coord(nd, L_bay, h_story, offset=0.25):
    # from node number, get the parent node it's attached to
    bot_nd = nd//100
    
    # get the bottom node's coordinates
    bot_x_coord = (bot_nd%10)*L_bay
    bot_y_coord = (bot_nd//10 - 1)*h_story
    
    # if last number is 1 or 2, brace connects nw
    # if last number is 3 or 4, brace connects ne
    goes_ne = [3, 4]
    if (nd%10 in goes_ne):
        x_offset = offset/2*L_bay/2
    else:
        x_offset = -offset/2*L_bay/2
    
    y_offset = offset/2 * h_story
    gp_x_coord = bot_x_coord + x_offset
    gp_y_coord = bot_y_coord + y_offset
    
    return(gp_x_coord, gp_y_coord)


def top_gp_coord(nd, L_bay, h_story, offset=0.25):
    # from node number, get the parent node it's attached to
    top_node = nd//100
    
    # extract their corresponding coordinates from the node numbers
    top_x_coord = (top_node%10 + 0.5)*L_bay
    top_y_coord = (top_node//10 - 1)*h_story
    
    # if last number is 1 or 5, brace connects se
    # if last number is 2 or 6, brace connects sw
    if (nd % 10)%2 == 0:
        x_offset = -offset/2*L_bay/2
    else:
        x_offset = offset/2*L_bay/2
    
    y_offset = -offset/2 * h_story
    gp_x_coord = top_x_coord + x_offset
    gp_y_coord = top_y_coord + y_offset
    
    return(gp_x_coord, gp_y_coord)
    

def mid_brace_coord(nd, L_bay, h_story, camber=0.001, offset=0.25):
    
    #######################################################
    # TODO: CHANGE THIS SECTION TO FIT YOUR NUMBERING SYSTEM
    # from mid brace number, get the corresponding top and bottom node numbers
    top_node = nd//100
    
    # extract their corresponding coordinates from the node numbers
    top_x_coord = (top_node%10 + 0.5)*L_bay
    top_y_coord = (top_node//10 - 1)*h_story
    
    # if the last number is 8, the brace connects sw
    # if the last number is 7, the brace connects se
    
    if (nd % 10)%2 == 0:
        bot_node = top_node - 10
        x_offset = offset/2 * L_bay/2
    else:
        bot_node = top_node - 9
        x_offset = - offset/2 * L_bay/2
    #######################################################
    
    
    # get the bottom node's coordinates
    bot_x_coord = (bot_node%10)*L_bay
    bot_y_coord = (bot_node//10 - 1)*h_story
    
    # effective length is 90% of the diagonal (gusset plate offset)
    br_x = abs(top_x_coord - bot_x_coord)
    br_y = abs(top_y_coord - bot_y_coord)
    L_eff = (1-offset)*(br_x**2 + br_y**2)**0.5
    
    # angle from horizontal up to brace vector
    from math import atan, asin, sin, cos
    theta = atan(h_story/(L_bay/2))
    
    # angle from the brace vector up to camber
    beta = asin(2*camber)
    
    # angle from horizontal up to camber
    gamma  = theta + beta
    
    # origin is bottom node, adjusted for gusset plate
    # offset is the shift (+/-) of the bottom gusset plate
    # terminus is top node, adjusted for gusset plate (gusset placed opposite direction)
    x_origin = bot_x_coord + x_offset
    x_terminus = top_x_coord - x_offset
    
    y_offset = offset/2 * h_story
    y_origin = bot_y_coord + y_offset
    y_terminus = top_y_coord - y_offset
    
    mid_x_coord = x_origin + L_eff/2 * cos(gamma)
    mid_y_coord = y_origin + L_eff/2 * sin(gamma)
    
    return(mid_x_coord, mid_y_coord)

# Building class

It's advantageous to make a class for our data, which allows us to internally store all of the design parameters and objects of the models (nodes, elements) without having to pass long lists of arguments to each function. We can also store all functions related to the building as methods that act on the Building object itself.

For example, the number_nodes method acts on the Building object by taking the Building and finding its number of bays and stories in order to add the lists of nodes and elements to the Building. Later on, we will add model and analyze as additional methods to the Building.

In [8]:
class Building:
    
    #########################################
    # INITIALIZING
    #########################################
    
    # import attributes as building characteristics from pd.Series
    
    # now the Series you see above is converted into a Building object
    # Values in the series are now attributes of the object
    def __init__(self, design):
        for key, value in design.items():
            setattr(self, key, value)
            
    #########################################
    # NUMBERING NODES AND ELEMENTS
    #########################################
    
    def number_nodes(self):
        
        # to interact with the attributes use the following below
        n_bays = int(self.num_bays)
        n_stories = int(self.num_stories)
        
        n_braces = np.round(n_bays/2.25) 
        
        # larger than 8 bays is not supported
        assert n_bays < 9
        
        ###### Main node system ######
        # Fixed nodes are 8xx, with xx being sequential from leftmost base
        #   898 and 899 are reserved for wall
        
        # All nodes are numbered xy, with x indicating floor number 
        #   1 at ground, n_stories+1 at roof
        # and y indicating column line
        #   0 at leftmost, n_bays at rightmost
        
        # Leaning column nodes are appended to the right at the same floor (n_bay+1)
        ##############################
        
        ##############################
        # YOUR CODE BELOW
        # Write your own implementation of the node numbering system
        # Store the results for each types of nodes by replacing the empty [] lists
        ##############################
        
        #main nodes
        main_nodes = []
        for i in range(1, n_stories+1):
            for j in range(0, n_bays+1):
                main_nodes.append(i*10+j)
                
        # base nodes
        base_id = 800
        base_nodes = []
        for b in range(0,n_bays+1):
            base_nodes.append(800+b)
        
        # wall nodes
        wall_nodes = [898, 899]
        
        # floor and leaning column nodes
        leaning_nodes = []
        for l in range(1, n_stories+1):
            leaning_nodes.append(l*10+n_bays+1)
        
        # EZ- brace nodes - point in the middle of the beam
        brace_nodes = []
        n_bstart = np.round(n_bays/2-n_braces/2)
        for f in range (2, n_stories+1):
            for b in range(int(n_bstart), int(n_bstart+n_braces)):
#TODO: Double check how the braces are numbered now
                brace_nodes.append(f*100+b*10+1)
                #brace_nodes.append(b*1000+(n_bays-1)*100+55)
        
        ###### Spring node system ######
        # Spring support nodes have the coordinates XYA, XY being the parent node
        # A is 6,7,8,9 for S,W,N,E respectively
        ################################
        
        # flatten list to get all nodes 
        # EZ- main nodes and leaning nodes for everyfloor minus top floor
        floor_nodes = []
        for i in range(1, n_bays+2):
            for j in range(1, n_stories):
                floor_nodes.append(i*10+j)
    
        # make south node if not on bottom floor
        
        # make west node if not on the leftmost column and bottom floor
        
        # make north node if not on top floor
        
        # make east node if not on rightmost column and bottom floor
        
        # repeat for leaning columns, only N-S
        lc_spr_nodes = []
        for s in range(len(leaning_nodes)-1):
            lc_spr_nodes.append(leaning_nodes[s]*10+6) #south
            lc_spr_nodes.append(leaning_nodes[s]*10+7) #west
            lc_spr_nodes.append(leaning_nodes[s]*10+8) #north
            #lc_spr_nodes.append(leaning_nodes[s]+"9") #east
        
        spring_nodes = []
        # make south node if not on bottom floor
        s_spr = [nd*10+6 for nd in floor_nodes if ((nd//10)%10 != 1)]
        # make west node if not on the leftmost column and bottom floor
        w_spr = [nd*10+7 for nd in floor_nodes if (nd%10) != 0 and ((nd//10)%10 != 1)]
        # make north node if not on top floor
        n_spr = [nd*10+8 for nd in floor_nodes if ((nd//10)%10 != n_stories+1)]
        # make east node if not on rightmost column and bottom floor
        e_spr = [nd*10+9 for nd in floor_nodes if (nd%10) != n_bays and ((nd//10)%10 != 1)]
        
        spring_nodes = s_spr + w_spr + n_spr + e_spr
        
        #brace related nodes
        # TODO: separate this into two lists: nodes connected to the beam (xxx3 and xxx4) vs. nodes connected to braces
        top_brace_related = []
        for b in brace_nodes:
            for n in range(1,6):
                top_brace_related.append(b*10+n)
                
        bottom_brace_related = []
        for f in floor_nodes:
            #only a bottom brace nodes if there are braces
            
            # TODO: this creates all 6 spring nodes for all bottom brace nodes
            # however, we only need all 6 nodes if the node has braces going both left and right out of it
            # so we should restrict the logic to create , for example, only 3, 4, 5 nodes if it is going to the right
            # and 1, 2, 6 nodes if it is going to the left
            
            
            if (f+10)*10+1 or ((f-1)+10)*10 in brace_nodes:
                for n in range(1,6):
                    bottom_brace_related.append(f*100+n)
            # TODO: separate this into two lists: 
            
        # TODO: add new nodes for shear tabs
        # if the main node xy is a part of the braced frame (within n_start and n_start+n_braced):
        # add a node xy0 (sits on the right) if the node xy is not the last column in the braced frame
        # add a node xy5 (sits on the left) if the node xy is not the first column in the braced frame
        
        ###### Element system ######
        # Elements all have an ID or "series" that all numbers of that type follows
        ################################
        
        # column elements, series 100
        col_id = 100
        # make column if not the top floor
        col_elems = []
        for c in range(len(floor_nodes)):
            col_elems.append(col_id+floor_nodes[c])
        
        # leaning column elements 
        lc_elems = []
        for l in range(1, n_stories):
            lc_elems.append(100+l*10+n_bays+1)
        
        # beam elements, series 200
        beam_id = 200 
        # make beam if not the last bay and not the bottom floor
        beam_elems = []
        for b in range(len(main_nodes)):
            beam_elems.append(beam_id+main_nodes[b])
        
        # truss elements, series 300
        truss_id = 300
        # make truss on the last bay for all floors
        truss_elems = []
        for t in range(1, n_stories+1):
            truss_elems.append(truss_id+t*10+n_bays+1)
        
        
        # diaphragm elements, series 400
        diaph_id = 400
        # make diaphragm if not the last bay on the bottom floor
        diaph_elems = []
        for d in range(1, n_bays+1):
            diaph_elems.append(diaph_id+10+d)
        
        # isolator elements, series 1000
        isol_id = 1000
        # make isolators above base nodes
        isol_elems = []
        for i in range(len(base_nodes)):
            isol_elems.append(isol_id+base_nodes[i])
        
        # spring elements, series 5000
        spring_id = 5000
        spring_elems = []
        for s in range(len(spring_nodes)):
            spring_elems.append(spring_id+spring_nodes[s])
            
        lc_spr_elems = []
        for l in range(len(lc_spr_nodes)):
            lc_spr_elems.append(spring_id+lc_spr_nodes[l])
            
        # spring for braces
        brace_spr_elems = []
        for s in range(len(top_brace_related)):
            brace_spr_elems.append(spring_id*10+top_brace_related)
        
        for s in range(len(bottom_brace_related)):
            brace_spr_elems.append(spring_id*10+bottom_brace_related)
        
        # wall elements, series 8000
        wall_id = 8000
        wall_elems = [8898, 8899]
        
        # write your own implementation for braces below!
        brace_id = 30000
        brace_elems = []
        for i in range(len(brace_nodes)):
            brace_elems.append(brace_id+brace_nodes[i]*10+1) #connecting bottom left
            brace_elems.append(brace_id+brace_nodes[i]*10+2) #connecting bottom right
        
        
        ###### Storing results ######
        # It's most efficient for us to store the lists of items nested in a dictionary of that type
        # The dictionaries themselves are now stored as attributes that we can easily access later on
        
        # example
        # Later when we access the objects, we simply get all of the floor nodes by using bldg.node_tags['floor_nodes']
        # Repeat the same for beam elements, e.g., with bldg.elem_tags['beam_elems']
        ################################
        self.node_tags = {
            'base': base_nodes,
            'wall': wall_nodes,
            'floor': floor_nodes,
            'leaning': leaning_nodes,
            'spring': spring_nodes,
            'lc_spring': lc_spr_nodes
            }
        
        self.elem_tags = {
            'col': col_elems, 
            'leaning': lc_elems, 
            'beam': beam_elems,
            'truss': truss_elems, 
            'diaphragm': diaph_elems, 
            'isolator': isol_elems, 
            'spring': spring_elems, 
            'lc_spring': lc_spr_elems, 
            'wall': wall_elems
            }
        
        self.elem_ids = {
            'col': col_id, 
            'leaning': col_id, 
            'beam': beam_id,
            'truss': truss_id, 
            'diaphragm': diaph_id, 
            'isolator': isol_id, 
            'spring': spring_id, 
            'lc_spring': spring_id, 
            'wall': wall_id,
            'base': base_id
            }
        
    #########################################
    # BUILDING THE MODEL IN OPENSEES
    #########################################
    
    def model_frame(self):
         # import OpenSees and libraries
        import openseespy.opensees as ops
        
        # remove existing model
        ops.wipe()

        # units: in, kip, s
        # dimensions
        inch    = 1.0
        ft      = 12.0*inch
        sec     = 1.0
        g       = 386.4*inch/(sec**2)
        kip     = 1.0
        ksi     = kip/(inch**2)
        
        L_bay = self.L_bay * ft     # ft to in
        h_story = self.h_story * ft
        w_floor = self.w_fl / ft    # kip/ft to kip/in
        p_lc = self.P_lc
        
        # set modelbuilder
        # x = horizontal, y = in-plane, z = vertical
        # command: model('basic', '-ndm', ndm, '-ndf', ndf=ndm*(ndm+1)/2)
        ops.model('basic', '-ndm', 3, '-ndf', 6)
        
        # masses of the nodes
        m_grav_inner = w_floor * L_bay / g
        m_grav_outer = w_floor * L_bay / 2 /g
        m_lc = p_lc / g
        
        # load for isolators vertical
        p_outer = sum(w_floor)*L_bay/2
        p_inner = sum(w_floor)*L_bay
        
        # nominal change
        L_beam = L_bay
        L_col = h_story
        
        ##############################
        # YOUR CODE BELOW
        # Write your own code to place the nodes
        ##############################
        
        # base nodes
        base_nodes = self.node_tags['base']
        
        # place base nodes here
        for b in range(len(base_nodes)):
            ops.node(base_nodes[b], b*L_bay*ft, 0, -1.0*ft)
            ops.fix(base_nodes[b], 1, 1, 1, 1, 1, 1)
    
        
        # TODO: edit the ops.fix conditions for nodes. Only walls and bases should be fully fixed.
        # remember to fix the nodes with
        # ops.fix(node_tag, 1, 1, 1, 1, 1, 1) for fully fixed
        # ops.fix(nd, 0, 1, 0, 1, 0, 1) for fully free
        
        # wall nodes (should only be two)
        n_bays = int(self.num_bays)
        n_floors = int(self.num_stories)
        
        # place wall nodes here
        wall_nodes = self.node_tags['wall']
        ops.node(wall_nodes[0], 0.0*ft, 0.0*ft, 0.0*ft)
        ops.fix(wall_nodes[0], 1, 1, 1, 1, 1, 1)
        ops.node(wall_nodes[1], n_bays*L_beam, 0.0*ft, 0.0*ft)
        ops.fix(wall_nodes[1], 1, 1, 1, 1, 1, 1)
        
        # place main nodes here
        # structure nodes
        # TODO: add mass onto the floor nodes
        # The m_grav_inner and m_grav_outer arrays have masses of the nodes according to their floors.
        # e.g. m_grav_inner[0] is the ground level mass for an interior node of the building, [1] is the first floor above etc
        # Use the command ops.mass(nd, m_nd, m_nd, m_nd, 1e-15, 1e-15, 1e-15) to add the mass
        floor_nodes = self.node_tags['floor']
        for nd in floor_nodes:
            ops.node(nd, (nd%10)*L_beam*ft, 0.0*ft, ((nd//10)%10-1)*h_story*ft)
            ops.fix(nd, 1, 1, 1, 1, 1, 1)
            # your mass code here!
            # negligible=1e-15
            # ops.mass(nd, m?, m?, m?, negligible, negligible, negligible)
            
        # leaning column nodes
        # TODO: the m_lc array contains the masses for the nodes on the leaning column. add them here correspondingly
        leaning_nodes = self.node_tags['leaning']
        for l in range(len(leaning_nodes)):
            ops.node(leaning_nodes[l], (n_bays+1)*L_bay*ft, 0, l*h_story*ft)
            ops.fix(leaning_nodes[l], 0, 1, 1, 1, 0, 1)
            # your mass code here!
            # negligible=1e-15
            # ops.mass(nd, m?, m?, m?, negligible, negligible, negligible)
            
        # SPECIAL CASE: we roller fix the bottom node of the leaning column
        # use ops.fix(nd, 0, 1, 1, 1, 0, 1)
        
        # place spring nodes here
        # spring nodes
        spring_nodes = self.node_tags['spring']
        for sn in spring_nodes:
            ops.node(sn, ((sn//10)%10)*L_beam*ft, 0.0*ft, ((sn//100)%10-1)*h_story*ft)
            ops.fix(sn, 1, 1, 1, 1, 1, 1)
            
        # leaning column spring nodes
        lc_spr_nodes = self.node_tags['lc_spring']
        for ln in spring_nodes:
            ops.node(ln, ((ln//10)%10)*L_beam*ft, 0.0*ft, ((ln//100)%10-1)*h_story*ft)
            ops.fix(ln, 1, 1, 1, 1, 1, 1)
        
        # EZ- brace nodes 
        brace_nodes = self.node_tags['brace_nodes']
        for bn in brace_nodes:
            ops.node(bn, ((bn//100)*10)*(L_bay+L_bay/2)*ft, 0*ft, ((bn//1000)%10-1)*h_story*ft)
            ops.fix(ln, 1, 1, 1, 1, 1, 1)
                     
        # TODO: Use the top, bot, and mid coord functions to place the following nodes:
        # Spring nodes for top brace nodes (e.g. 3111, 3115, 3112, 3116)
        # Spring nodes for bottom brace nodes (e.g. 2101, 2103, 2104, 2102)
        # Mid-brace nodes (your system)
        
        # sample call: x_coord, z_coord = mid_brace_coord(nd, L_bay, h_story)
        # ops.node(nd, x_coord, 0.0, z_coord)
        
        print('Nodes placed.')
        
################################################################################
# define materials
################################################################################

        # define your tags here. Tags are numbers for materials. There is no system, except to not reuse a tag.
    
        # define material: steel
        Es  = 29000*ksi     # initial elastic tangent
        nu  = 0.2          # Poisson's ratio
        Gs  = Es/(1 + nu) # Torsional stiffness modulus
        J   = 1e10          # Set large torsional stiffness
    
        # TODO: make an elastic stiff material
        # Make an elastic material with the stiffness of steel (Es)
        # https://openseespydoc.readthedocs.io/en/latest/src/ElasticUni.html
        # your code here. 
        
        # TODO: make an elastic "ghost" material with very small stiffness (100.0 ksi)
        # https://openseespydoc.readthedocs.io/en/latest/src/ElasticUni.html
        # your code here. 
        
        # TODO: make an elastic material in the torsion direction with stiffness J
        # your code here
        
        # TODO: make a Steel02 material with the following properties
        # https://openseespydoc.readthedocs.io/en/latest/src/steel02.html
        Fy  = 50*ksi        # yield strength
        b   = 0.1           # hardening ratio
        R0 = 15
        cR1 = 0.925
        cR2 = 0.15
        # your code here
        
        # TODO: 'wrap' your steel material with a fatigue material to allow it to weaken after repeated cycling
        # https://openseespydoc.readthedocs.io/en/latest/src/Fatigue.html
        # ops.uniaxialMaterial('Fatigue', your_new_material_tag, your_steel02_material_tag)
        
        # Gusset plate 
        W_w = (L_gp**2 + L_gp**2)**0.5
        L_avg = 0.75* L_gp
        t_gp = 1.375*inch
        Fy_gp = 50*ksi
        
        My_GP = (W_w*t_gp**2/6)*Fy_gp
        K_rot_GP = Es/L_avg * (W_w*t_gp**3/12)
        b_GP = 0.01
        
        # TODO: make a Steel02 material with rotational properties of a gusset plate.
        # It has an 'Fy' strength of My_GP, an 'E' stiffness of value K_rot_GP, a post-yielding ratio of b_GP, 
        # and the remaining parameters is the same as your steel
        # your code here

## Test out the building

First, initialize it by getting the parameters from the Series. Then number the nodes.

In [9]:
cbf_bldg = Building(cbf_params)

cbf_bldg.number_nodes()

cbf_bldg.node_tags
cbf_bldg.elem_tags
cbf_bldg.elem_ids


TypeError: unsupported operand type(s) for +: 'int' and 'list'

## Try modeling the frame

We start with just the nodes for now.

In [12]:
cbf_bldg.model_frame()

Domain::addNode - node with tag 216already exists in model


OpenSeesError: See stderr output